In [27]:
## LOAD MODEL
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3:latest", temperature=0)

## TEST MESSAGING
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content="Hi! I'm Bob")])


AIMessage(content="Hi Bob! It's nice to meet you. Is there something on your mind that you'd like to chat about, or are you just looking for some friendly conversation? I'm here to listen and help if I can.", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-03-09T16:04:30.7290175Z', 'done': True, 'done_reason': 'stop', 'total_duration': 23177435300, 'load_duration': 10341838700, 'prompt_eval_count': 15, 'prompt_eval_duration': 2039000000, 'eval_count': 46, 'eval_duration': 10795000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-24543c37-4421-4427-8ab8-b13419412e83-0', usage_metadata={'input_tokens': 15, 'output_tokens': 46, 'total_tokens': 61})

In [28]:
## SYSTEM PROMPT

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a travel planner. Ask user about travel preferences and use this information to propose a travel plan.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [29]:
## MESSAGE TRIMMING - how much past messages are remembered

from langchain_core.messages import AIMessage, SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)


In [30]:
## PERSISTENCE
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# DEFINE CLASS INPUTS
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

# Define a new graph
workflow = StateGraph(state_schema=State)


# Define the function that calls the model
def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = llm.invoke(prompt)
    return {"messages": [response]}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)





In [31]:
### TEST EXAMPLE

# Add config to tell different chats apart
config = {"configurable": {"thread_id": "abc123"}}

query = "Can you recommend me a trip?"
language = "English"

messages = [HumanMessage(content='I like beaches, adventure and sunbathing')]


input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()  # output contains all messages in state

# query = "What is my name?"
#
# input_messages = [HumanMessage(query)]
# output = app.invoke({"messages": input_messages}, config)
# output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sounds like you're looking for a tropical getaway with a mix of relaxation and excitement!

Considering your love for beaches, adventure, and sunbathing, I'd like to propose a trip to the beautiful island of Maui, Hawaii.

Maui offers:

1. Stunning beaches: You'll find world-class beaches like Ka'anapali Beach, Wailea Beach, and Hanauma Bay, perfect for swimming, snorkeling, or simply soaking up the sun.
2. Adventure activities:
	* Snorkeling or scuba diving to explore Maui's vibrant marine life
	* Whale watching (seasonal) to spot humpback whales in their natural habitat
	* Ziplining or helicopter tours for a bird's-eye view of the island's lush landscapes
3. Sunbathing and relaxation:
	* Unwind on the beach with a good book and a refreshing cocktail
	* Treat yourself to a rejuvenating spa day at one of Maui's many luxurious resorts

To make the most of your trip, I recommend staying in Ka'anapali or Wai